
## Import dependencies

In [1]:
import os
import time

import pandas
import torch

from proselflc.trainer.trainer_cnn_vision_derivedgrad import Trainer

import pprint
print = pprint.PrettyPrinter(indent=4).pprint

## Add params configurations

In [2]:
params = {}
params.update(
    {
        "data_name": "cifar100",
        "num_classes": 100,  # 1000
        "device": "gpu",
        #
        "num_workers": 8,
        #
        "counter": "iteration",
        "classes_per_batch": 8,
        #
        # To get deterministic results, please uncomment the line below
        # to set seed parameter.
        # "seed": 123,
    }
)

# data
params["symmetric_noise_rate"] = 0.4

# network
params["network_name"] = "shufflenetv2"

# for the demo purpose, I set the total epochs to be small.
params["total_epochs"] = 100
params["eval_interval"] = 500 # iterations

# batch
params["batch_size"] = 128
params["sampler"] = "BalancedBatchSampler"
params["classes_per_batch"] = 64

# learning rate
params["lr"] = 0.2
params["weight_decay"] = 1e-3
params["lr_scheduler"] = "WarmupMultiStepSchedule"
params["warmup_epochs"] = 0
params["milestones"] = [20000, 30000]

# optimisation
params["momentum"] = 0.9
params["batch_accumu_steps"] = 10
params["gamma"] = 0.1

# loss settings
params["loss_name"] = "labelsmoothing"
params["logit_soften_T"] = None
params["epsilon"] = 0.25 # can also try 0.2, 0.5, etc

print(params)

{   'batch_accumu_steps': 10,
    'batch_size': 128,
    'classes_per_batch': 64,
    'counter': 'iteration',
    'data_name': 'cifar100',
    'device': 'gpu',
    'epsilon': 0.25,
    'eval_interval': 500,
    'gamma': 0.1,
    'logit_soften_T': None,
    'loss_name': 'labelsmoothing',
    'lr': 0.2,
    'lr_scheduler': 'WarmupMultiStepSchedule',
    'milestones': [20000, 30000],
    'momentum': 0.9,
    'network_name': 'shufflenetv2',
    'num_classes': 100,
    'num_workers': 8,
    'sampler': 'BalancedBatchSampler',
    'symmetric_noise_rate': 0.4,
    'total_epochs': 100,
    'warmup_epochs': 0,
    'weight_decay': 0.001}


## Create the folder to store intermediate and final results
* First, run `sudo mkdir /home/proselflc_experiments/ && sudo chmod -R 777 /home/proselflc_experiments/` so that so you have the write permission.
* Or set `WORK_DIR = "/home/your_username/proselflc_experiments/"`

In [3]:
WORK_DIR = "/home/proselflc_experiments/"
#
# use the time as a unique experiment identifier
dt_string = time.strftime("%Y%m%d-%H%M%S")
summary_writer_dir = (
    params["loss_name"]
    + "_"
    + dt_string
)
params["summary_writer_dir"] = (
    WORK_DIR
    + "/"
    + params["data_name"]
    + "_symmetric_noise_rate_"
    + str(params["symmetric_noise_rate"])
    + "/"
    + params["network_name"]
    + "/"
    + summary_writer_dir
)
if not os.path.exists(params["summary_writer_dir"]):
    os.makedirs(params["summary_writer_dir"])

## Init the trainer, and store the params configurations
For each experiment, we have one unique result folder to store the params configurations and learning curves.
Therefore, you can revisit any specific experiment whenever you need without losing any details.

In [4]:
trainer = Trainer(params=params)
# params["milestones"] was a list of integers, we convert it to a string before sinking.
params["milestones"] = str(params["milestones"])
dataframe = pandas.DataFrame(params, index=[0])
dataframe.to_csv(
    params["summary_writer_dir"] + "/params.csv",
    encoding="utf-8",
    index=False,
    sep="\t",
    mode="w",  #
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Run the trainer and save the final model

In [5]:
trainer.train()
torch.save(
    trainer.network,
    params["summary_writer_dir"] + "/model.pt",
)
print(
    "The experiment is finished with details sinked in {}".format(
        params["summary_writer_dir"]
    )
)

Evaluating Network.....
Iteration= (500,2)/(39100, 100), lr=0.2000, batch_mean_epsilon=-1.0000, valid batch size=-1.0000, batch_mean_gtrust=-1.0000, batch_mean_etrust=-1.0000
clean_test: Loss= 4.3563, Accuracy= 0.0406, Entropy= 0.9717, Max_p= 0.0334,
noisy_subset: Loss= 4.7029, Accuracy= 0.0106, Entropy= 0.9767, Max_p= 0.0295,
clean_subset: Loss= 4.3600, Accuracy= 0.0442, Entropy= 0.9762, Max_p= 0.0298,
cleaned_noisy_subset: Loss= 4.3789, Accuracy= 0.0422, Entropy= 0.9767, Max_p= 0.0295,
Evaluating Network.....
Iteration= (1000,3)/(39100, 100), lr=0.2000, batch_mean_epsilon=-1.0000, valid batch size=-1.0000, batch_mean_gtrust=-1.0000, batch_mean_etrust=-1.0000
clean_test: Loss= 4.0847, Accuracy= 0.0945, Entropy= 0.9566, Max_p= 0.0463,
noisy_subset: Loss= 4.7456, Accuracy= 0.0095, Entropy= 0.9623, Max_p= 0.0420,
clean_subset: Loss= 4.0958, Accuracy= 0.0923, Entropy= 0.9615, Max_p= 0.0427,
cleaned_noisy_subset: Loss= 4.1277, Accuracy= 0.0847, Entropy= 0.9623, Max_p= 0.0419,
Evaluating Ne

KeyboardInterrupt: 